In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from pathlib import Path
import math

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [ ]:
from mpl_utils import MPLAdjutant
adj = MPLAdjutant()
adj.set_defaults()

In [ ]:
import gpytorch
import botorch

In [ ]:
from easyBO import gp, bo
from easyBO.logger import mode

# Very simple example from the BoTorch docs using `easyBO`

See [here](https://botorch.org/v/0.1.0/tutorials/fit_model_with_torch_optimizer). In this simple example, we do the following:
1. Initialize a single task GP regressor from dummy training data
2. Assume homoscedastic noise
3. Train the GP hyperparameters
4. Plot the results

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

# use regular spaced points on the interval [0, 1]
train_x = torch.linspace(0, 1, 15)

# training data needs to be explicitly multi-dimensional
train_x = train_x.unsqueeze(1)

# sample observed values and add some synthetic noise
train_y = torch.sin(train_x * (2 * math.pi)) + 0.15 * torch.randn_like(train_x)

# Testing grid
grid = torch.linspace(0, 1, 101)

Get the initial model conditioned on the training data, and run inference on the un-optimized GP, just to see what it looks like.

In [ ]:
model = gp.get_gp(train_x=train_x, train_y=train_y, gp_type="regression")
pre_fitting_infer = gp.infer(model=model, grid=grid)

In [ ]:
for param in model.named_parameters():
    print(param)

Now, optimize the hyper-parameters (by default, this is just a kernel of the form `Const x RBF`.

In [ ]:
losses = gp.train_gp_(model=model, training_iter=200)

In [ ]:
for param in model.named_parameters():
    print(param)

Run inference again on the trained model.

In [ ]:
post_fitting_infer = gp.infer(model=model, grid=grid)

Plot them both.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2), sharey=True, sharex=True)

ax = axs[0]
adj.set_grids(ax, grid=False)
ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, pre_fitting_infer["mean"], "r-")
ax.fill_between(grid.squeeze(), pre_fitting_infer['mean-2sigma'], pre_fitting_infer['mean+2sigma'], alpha=0.2, color="red", linewidth=0)

ax = axs[1]
adj.set_grids(ax, grid=False)
ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, post_fitting_infer["mean"], "r-")
ax.fill_between(grid.squeeze(), post_fitting_infer['mean-2sigma'], post_fitting_infer['mean+2sigma'], alpha=0.2, color="red", linewidth=0)

axs[0].set_ylabel(r"$f(x)$")

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(r"$x$", labelpad=20)

plt.subplots_adjust(wspace=0.1)
plt.show()

# Another example (this one is custom)

Another very simple example, but this one uses a custom function which is a bit trickier than the one above. That said, the procedure we take is identical.

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

x1 = np.linspace(0, 0.5, 10)
x3 = np.linspace(0.75, 1, 10)

train_x = torch.FloatTensor(np.concatenate([x1, x3])).reshape(-1, 1)
train_y = 5*train_x**2 + np.sin(train_x * 40) / 3 + torch.FloatTensor(np.random.normal(scale=0.1, size=train_x.shape))
train_y = train_y

grid = np.linspace(-2, 3, 1000).reshape(-1, 1)

In [ ]:
model = gp.get_gp(train_x=train_x, train_y=train_y, gp_type="regression")
pre_fitting_infer = gp.infer(model=model, grid=grid)

In [ ]:
losses = gp.train_gp_(model=model)
post_fitting_infer = gp.infer(model=model, grid=grid)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2), sharey=True, sharex=True)

ax = axs[0]
adj.set_grids(ax, grid=False)
ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, pre_fitting_infer["mean"], "r-")
ax.fill_between(grid.squeeze(), pre_fitting_infer['mean-2sigma'], pre_fitting_infer['mean+2sigma'], alpha=0.2, color="red", linewidth=0)

ax = axs[1]
adj.set_grids(ax, grid=False)
ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, post_fitting_infer["mean"], "r-")
ax.fill_between(grid.squeeze(), post_fitting_infer['mean-2sigma'], post_fitting_infer['mean+2sigma'], alpha=0.2, color="red", linewidth=0)

axs[0].set_ylabel(r"$f(x)$")

ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(r"$x$", labelpad=20)

plt.subplots_adjust(wspace=0.1)
plt.show()

# Bayesian optimization example

Follow along [here](https://botorch.org/tutorials/compare_mc_analytic_acquisition). This is a simple boilerplate example using a tutorial. We won't be visualizing/testing anything specific.

In [ ]:
from botorch.test_functions import Hartmann
neg_hartmann6 = Hartmann(dim=6, negate=True)

In [ ]:
train_x = torch.rand(10, 6)
train_y = neg_hartmann6(train_x).unsqueeze(-1)

In [ ]:
model = gp.get_gp(train_x=train_x, train_y=train_y, gp_type="regression")

In [ ]:
loss = gp.train_gp_(model=model, training_iter=1000)

We'll use the `ExpectedImprovement` acquisition function for now. This requires `best_value`. To start, we'll use the standard analytic `EI`, which is implemented for us in `bo.ask`.

In [ ]:
best_value = train_y.max()

In [ ]:
torch.manual_seed(0) # to keep the restart conditions the same
with mode(debug=True):
    asked_new_point = bo.ask(
        model=model,
        bounds=[(0, 1) for _ in range(6)],
        acquisition_function="EI",
        acquisition_function_kwargs={"best_f": best_value},
        optimize_acqf_kwargs={"q": 1, "num_restarts": 20, "raw_samples": 100}
    )

In [ ]:
asked_new_point

We can then use a Monte Carlo sampler implemented in `botorch`. These can seamlessly be passed to the `bo.ask` function.

In [ ]:
from botorch.acquisition import qExpectedImprovement
from botorch.sampling import SobolQMCNormalSampler

In [ ]:
sampler = SobolQMCNormalSampler(num_samples=500, seed=0, resample=False)
torch.manual_seed(seed=0) # to keep the restart conditions the same

In [ ]:
asked_mc_point = bo.ask(
    model=model,
    bounds=[(0, 1) for _ in range(6)],
    acquisition_function=qExpectedImprovement,
    acquisition_function_kwargs={"best_f": best_value, "sampler": sampler},
    optimize_acqf_kwargs={"q": 1, "num_restarts": 20, "raw_samples": 100}
)

In [ ]:
asked_mc_point

Note that for the same random states the analytic and Monte Carlo results are in perfect agreement (and generally, they are always in _almost_ perfect agreement). Finally, we demonstrate the joint optimization of the acquisition function, the true power of `botorch`, where we can actually draw multiple samples at a time.

In [ ]:
sampler = SobolQMCNormalSampler(num_samples=500, seed=0, resample=False)
torch.manual_seed(seed=0) # to keep the restart conditions the same

In [ ]:
asked_mc_point = bo.ask(
    model=model,
    bounds=[(0, 1) for _ in range(6)],
    acquisition_function=qExpectedImprovement,
    acquisition_function_kwargs={"best_f": best_value, "sampler": sampler},
    optimize_acqf_kwargs={"q": 2, "num_restarts": 20, "raw_samples": 512}
)

In [ ]:
asked_mc_point

These sampled points are those which will _jointly_ optimize the acquisition function. In other words, we can "ask" for multiple points at a time!

# A practical active learning example

Often we just want to "plug-and-play" with active learning to make e.g. experiments more efficient. It doesn't seem that `botorch` has simple active learning implementations, so we wrote our own. These are small modifications of the `UpperConfidenceBound` classes, where we basically set `beta -> infinity`.

In [ ]:
np.random.seed(123)
x1 = np.linspace(0, 0.2, 20)
x4 = np.linspace(1.4, 1.6, 20)
train_x = torch.FloatTensor(np.concatenate([x1, x4])).reshape(-1, 1)
train_y = 5*train_x**2 + np.sin(train_x * 50) / 3 + torch.FloatTensor(np.random.normal(scale=0.1, size=train_x.shape))
train_y = train_y
grid = np.linspace(-2, 3, 1000).reshape(-1, 1)

In [ ]:
model = gp.get_gp(train_x=train_x, train_y=train_y, gp_type="regression")
pred = gp.infer(model=model, grid=grid)

For consistency with the other examples, here's the un-optimized GP conditioned on the training data.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)

plt.show()

In [ ]:
losses = gp.train_gp_(model=model, training_iter=1000)

In [ ]:
pred = gp.infer(model=model, grid=grid)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)

plt.show()

## Ask

After optimization, we can play around with the `MaxVar` and `qMaxVar`policies. Lets start with the simpler analytic `MaxVar` policy.

In [ ]:
import botorch

In [ ]:
bounds = [(-0.2, 1.9)]
pt = bo.ask(
    model=model,
    bounds=bounds,
    acquisition_function="MaxVar",
    acquisition_function_kwargs=dict(),
    optimize_acqf_kwargs={"q": 1, "num_restarts": 20, "raw_samples": 512},
    weight=None
)

In [ ]:
observed_pred = pred["observed_pred"]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(3, 2), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)

ax = axs[0]
ax.plot(grid, observed_pred.variance.detach().numpy(), color="blue")
ax.axvline(pt.item(), linestyle="--", color="black")
ax.axvline(bounds[0][0], linestyle="--", linewidth=0.5, color="black")
ax.axvline(bounds[0][1], linestyle="--", linewidth=0.5, color="black")
adj.set_grids(ax, grid=False)
ax.set_ylabel(r"$\sigma^2$")

ax = axs[1]
ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)
ax.axvline(pt.item(), linestyle="--", color="black")
ax.axvline(bounds[0][0], linestyle="--", linewidth=0.5, color="black")
ax.axvline(bounds[0][1], linestyle="--", linewidth=0.5, color="black")
adj.set_grids(ax, grid=False)
ax.set_ylabel(r"$f(x)$")
ax.set_xlabel(r"$x$")

plt.show()

Now however, we can _jointly_ optimize the variance. Say that we can run 2 experiments in parallel. It might not be the case that the point chosen when just doing a sequential experiment is the same as one of the two points that are jointly optimizing.

In [ ]:
pts = bo.ask(
    model=model,
    bounds=[(-0.2, 1.5)],
    acquisition_function="qMaxVar",  # note the qMaxVar
    acquisition_function_kwargs=dict(),
    optimize_acqf_kwargs={"q": 2, "num_restarts": 20, "raw_samples": 512},  # Note q==2
    # weight=lambda x: torch.abs(x)
)

In [ ]:
pts

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(3, 2), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)

ax = axs[0]
ax.plot(grid, observed_pred.variance.detach().numpy(), color="blue")
for pt in pts:
    ax.axvline(pt.item(), linestyle="--", color="black")
ax.axvline(bounds[0][0], linestyle="--", linewidth=0.5, color="black")
ax.axvline(bounds[0][1], linestyle="--", linewidth=0.5, color="black")
adj.set_grids(ax, grid=False)
ax.set_ylabel(r"$\sigma^2$")

ax = axs[1]
ax.scatter(train_x.numpy(), train_y.numpy(), color="black", s=0.5)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)
for pt in pts:
    ax.axvline(pt.item(), linestyle="--", color="black")
ax.axvline(bounds[0][0], linestyle="--", linewidth=0.5, color="black")
ax.axvline(bounds[0][1], linestyle="--", linewidth=0.5, color="black")
adj.set_grids(ax, grid=False)
ax.set_ylabel(r"$f(x)$")
ax.set_xlabel(r"$x$")

plt.show()

Indeed, these are not the same points.

## Tell

In [ ]:
new_x = torch.tensor(np.array([-1.5, -1.0]).reshape(-1, 1)).float()
new_y = torch.tensor(np.array([5.0, 4.0])).float()

In [ ]:
new_model = gp.tell(model=model, new_x=new_x, new_y=new_y)

In [ ]:
new_x, new_y = gp.get_training_data(model=model)

In [ ]:
pred = gp.infer(model=new_model, grid=grid)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

ax.scatter(new_model.train_inputs[0].detach().numpy(), new_model.train_targets.detach().numpy(), color="black", s=3)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)

plt.show()

Using the new model (and note, we've told it about the new data), we can then refine the hyperparameters further.

In [ ]:
with gpytorch.settings.debug(False):
    losses = gp.train_gp_(model=new_model, training_iter=1000)

In [ ]:
pred = gp.infer(model=new_model, grid=grid)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

ax.scatter(new_model.train_inputs[0].detach().numpy(), new_model.train_targets.detach().numpy(), color="black", s=3)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)

plt.show()

We often want to then improve the model with the next-sampled point. We can do this in two flavors.
- Recondition the GP, but don't revise the hyperparameters
- Recondition the GP, but do re-train the hyperparamters

# A toy experiment

Let's now consider the holy grail: we want to run an experiment according to some policy (e.g. to find the best interpolating function which is very certain everywhere, or to maximize some quantity). Let's do this on a toy source of truth that is easy to visualize. We'll use the same function as before:

In [ ]:
def truth(x):
    if isinstance(x, (float, int)):
        x = torch.tensor(x, dtype=torch.float)
    return 5*x**2 + torch.sin(x * 50) / 3 + torch.FloatTensor(np.random.normal(scale=0.2, size=x.shape))

In [ ]:
np.random.seed(123)

x1 = np.linspace(0, 0.2, 2)
x4 = np.linspace(1.4, 1.6, 2)
train_x = torch.FloatTensor(np.concatenate([x1, x4])).reshape(-1, 1)
train_y = truth(train_x)
grid = np.linspace(-2, 3, 1000).reshape(-1, 1)

We begin by initializing a model on minimal training data, as usual, and then training it.

In [ ]:
model = gp.get_gp(train_x=train_x, train_y=train_y, gp_type="regression")
_ = gp.train_gp_(model=model, training_iter=1000)
pred = gp.infer(model=model, grid=grid)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

ax.scatter(train_x.detach().numpy(), train_y.detach().numpy(), color="black", s=3)
ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)

plt.show()

Consider that we wish to do simple active learning. In that case, we want to create a loop, where the at each instance of the loop we "ask" for a new point, query the source of `truth`, and add that data back to the training set.

In [ ]:
def silly_weight(x):
    where = (x > 0) & (x < 1)
    weights = torch.zeros(x.shape)
    weights[where] = 1
    return weights

In [ ]:
torch.manual_seed(123)

NLOOP = 10

for ii in range(NLOOP):
    candidate = bo.ask(
        model=model,
        bounds=[(-2, 3)],
        acquisition_function="qMaxVar",
        optimize_acqf_kwargs={
            "q": 1,
            "num_restarts": 5,
            "raw_samples": 20,
        },
        weight=None
    )
    model = gp.tell(model=model, new_x=candidate, new_y=truth(candidate))

In [ ]:
loss = gp.train_gp_(model=model, training_iter=1000)

In [ ]:
pred = gp.infer(model=model, grid=grid)

In [ ]:
new_x, new_y = gp.get_training_data(model=model)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))


ax.plot(grid, pred["mean"], "r-")
ax.fill_between(grid.squeeze(), pred['mean-2sigma'], pred['mean+2sigma'], alpha=0.5)
ax.scatter(new_x, new_y, color="black", s=3)

plt.show()

# 2d input data

In [ ]:
def grids_to_coordinates(grids):
    x = np.meshgrid(*grids)
    return np.array([xx.flatten() for xx in x]).T

In [ ]:
np.random.seed(127)
N = 100
M = 150
idx = np.random.choice([xx for xx in range(N*M)], 20, replace=False)
idx.sort()

grid_x = np.linspace(-4, 5, N)
grid_y = np.linspace(-5, 4, M)

# Feature data
g1, g2 = np.meshgrid(grid_x, grid_y)
X = np.array([g1.flatten(), g2.flatten()]).T
X = X[idx, :]

X_original = X.copy()

# alpha = (np.linspace(-2, 2, N**2)[idx])**2 * 0  # Noise/uncertainty
alpha = np.array([1e-5 for _ in range(len(X))])

def func(x, y):
    return (1 - x / 3. + x ** 5 + y ** 5) * np.exp(-x ** 2 - y ** 2) + np.exp(-(x - 2)**2 - (y + 4)**2)

def truth(X):
    x = X[:, 0]
    y = X[:, 1]
    return func(x, y)

def truth_meshgrid(x, y):
    x = x.reshape(-1, 1)
    y = y.reshape(1, -1)
    return func(x, y)
    

y = truth(X)  # Target data
y = y.reshape(-1, 1)
train_x = X
train_y = y

In [ ]:
grid = grids_to_coordinates([grid_x, grid_y])
z = truth_meshgrid(grid_x, grid_y)
z_min = -np.abs(z).max()
z_max = np.abs(z).max()

In [ ]:
model = gp.get_gp(train_x=train_x, train_y=train_y, gp_type="regression")
_ = gp.train_gp_(model=model, training_iter=1000)

In [ ]:
pred = gp.infer(model=model, grid=grid)

In [ ]:
mu = pred["mean"].reshape(M, N)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3), sharey=True, sharex=True)

ax = axs[0]
c = ax.imshow(
    z.T, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
    interpolation ='nearest', origin ='lower'
)
adj.set_grids(ax, grid=False)
ax.set_title("Function")

ax = axs[1]
c = ax.imshow(
    mu, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
    interpolation ='nearest', origin ='lower'
)
adj.set_grids(ax, grid=False)
ax.scatter(train_x[:, 0], train_x[:, 1], s=0.3, color="black")
# ax.scatter(X_original[:, 0], X_original[:, 1], s=0.3, color="blue")
ax.set_title("GP")

plt.show()

In [ ]:
def weight(x):
    # x should be of shape [..., 2]
    # lets weight the y-value a lot if its order of magnitude is high
    return torch.abs(x[:, 1])

In [ ]:
torch.manual_seed(123)

NLOOP = 20

for ii in range(NLOOP):
    candidate = bo.ask(
        model=model,
        bounds=[(-4, 5), (-5, 4)],
        acquisition_function="qUpperConfidenceBound", # acquisition_function="MaxVar",
        acquisition_function_kwargs={"beta": 0.1},
        optimize_acqf_kwargs={
            "q": 1,
            "num_restarts": 5,
            "raw_samples": 20,
        },
        weight=weight
    )
    model = gp.tell(model=model, new_x=candidate, new_y=truth(candidate).reshape(1, 1))

In [ ]:
loss = gp.train_gp_(model=model, training_iter=1000)

In [ ]:
pred = gp.infer(model=model, grid=grid)

In [ ]:
mu = pred["mean"].reshape(M, N)

In [ ]:
new_x, new_y = gp.get_training_data(model=model)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3), sharey=True, sharex=True)

ax = axs[0]
c = ax.imshow(
    z.T, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
    interpolation ='nearest', origin ='lower'
)
adj.set_grids(ax, grid=False)
ax.set_title("Function")

ax = axs[1]
c = ax.imshow(
    mu, cmap='rainbow', vmin=z_min, vmax=z_max,
    extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
    interpolation ='nearest', origin ='lower'
)
adj.set_grids(ax, grid=False)
ax.scatter(new_x[:, 0], new_x[:, 1], s=0.3, color="black")
# ax.scatter(X_original[:, 0], X_original[:, 1], s=0.3, color="blue")
ax.set_title("GP")

plt.show()